# TD Ameritrade API  
## Price History and Search Instruments

Install modules

In [125]:
import sys
# !{sys.executable} -m pip install

Load modules

In [126]:
from auth import TDClient

In [127]:
import os
import requests
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import time
from datetime import datetime, date, timedelta
from cassandra.cluster import Cluster

Set project directory and paths to credentials and tokens files

In [128]:
base_path = os.path.join(os.path.expanduser('~'), 'bakery')
creds_path = os.path.join(base_path, 'td_ameritrade/creds.json')
token_path = os.path.join(base_path, 'td_ameritrade/tokens.json')
nasdaq_path = os.path.join(base_path, 'td_ameritrade/data/nasdaq.csv')

Get authentication object

In [129]:
td_client = TDClient()
td_client.refresh_tokens()
td_auth = td_client.get_auth()

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


In [130]:
cluster = Cluster()
session = cluster.connect()

Get list of symbols from Cassandra table: bakery.symbols

In [131]:
def list_factory(colnames, rows):
    return [row[0] for row in rows]
session.row_factory = list_factory

query = 'select symbol from bakery.symbols;'
symbols = session.execute(query, timeout=None)
symbols = [symbol for symbol in symbols]

Get price history for one symbol

In [132]:
# params={
#     'periodType': 'year',
#     'period': 20,
#     'frequencyType': 'daily',
#     'frequency': 1
# }

# r = requests.get(
#         f'https://api.tdameritrade.com/v1/marketdata/MSFT/pricehistory',
#         auth=td_auth,
#         params=params
#     )

In [133]:
# for candle in r.json()['candles']:
#     dt = datetime.fromtimestamp(candle['datetime']/1000).strftime('%Y-%m-%d')
#     values = (
#         dt,
#         'MSFT',
#         candle['open'],
#         candle['high'],
#         candle['low'],
#         candle['close'],
#         candle['volume']
#     )
#     print(values)

Get price history for list of symbols

In [134]:
max_calls_per_minute = 100
num_calls = 0
start_time = time.time()
calls_per_minute = 0

for symbol in (pbar := tqdm(symbols)):
    pbar.set_description(f'Symbol: {symbol}, Requests/Min: {calls_per_minute}')
    empty_response = False
    
    params={
        'periodType': 'year',
        'period': 20,
        'frequencyType': 'daily',
        'frequency': 1
    }
    
    
    if time.time() > td_client.token['expires_at']:
        td_client = TDClient()
        td_client.refresh_tokens()
        td_auth = td_client.get_auth()

    r = requests.get(
            f'https://api.tdameritrade.com/v1/marketdata/{symbol}/pricehistory',
            auth=td_auth,
            params=params
        )
    
    num_calls += 1
    
    try:
        candles = r.json()['candles']
    except ValueError:
        empty_response = True
    
    if not empty_response:
        for candle in r.json()['candles']:
            dt = datetime.fromtimestamp(candle['datetime']/1000).strftime('%Y-%m-%d')
            columns = (
                'dt', 'symbol', 'open_price', 'high_price', 'low_price', 'close_price', 'total_volume'
            )
            values = (
                dt,
                symbol,
                candle['open'],
                candle['high'],
                candle['low'],
                candle['close'],
                candle['volume']
            )
            
            null_idxs = [i for i, value in enumerate(values) if value == 'NaN']
            non_null_values = tuple([value for value in values if value != 'NaN'])
            non_null_columns = '(' + ', '.join([column for i, column in enumerate(columns) if i not in null_idxs]) + ')'
            
            query = f'''
                INSERT INTO bakery.prices_daily {non_null_columns}
                VALUES {non_null_values}
            '''
            
            session.execute(query)
    
    elapsed_time = (time.time() - start_time) / 60
    calls_per_minute = num_calls / elapsed_time
    if calls_per_minute > max_calls_per_minute:
        time.sleep(1)

Symbol: CHI, Requests/Min: 33.31173514126262:  12%| | 998/8162 [29:57<3:28:10,  

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Symbol: NBSE, Requests/Min: 33.41975362812427:  25%|▏| 2004/8162 [59:57<3:26:06,

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Symbol: BAC^M, Requests/Min: 33.20119428943188:  37%|▎| 2987/8162 [1:29:57<1:52:

...

Symbol: KUKE, Requests/Min: 33.99347587329734:  87%|▊| 7138/8162 [3:29:58<25:14,

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Symbol: XRX, Requests/Min: 34.0006017745085: 100%|▉| 8160/8162 [3:59:59<00:03,  

Access token expired. Calling get_token_from_refresh_token()
Access token has been successfully updated


Symbol: ZETA, Requests/Min: 33.99557306126515: 100%|█| 8162/8162 [4:00:04<00:00,


In [124]:
values[6] - 2147483647

620943553

In [ ]:
2147483647

In [121]:
for candle in r.json()['candles']:
    if candle['volume'] == 2768427200:
        print(candle['volume'])

2768427200


In [103]:
td_client

In [104]:
td_client.token

{'access_token': '2lJJY24XLNL0ToP83rYzURFPbPtcdvcjnBWgGfYigjwACTiJSdRQgdMu8kGONsfYPpgsYnv7lQ2ess33p/wo68FmXwwFAThFYUvPDzfhhLIDAfDiQGCiEzMwNwXD4jzv4WiTZuWDQHzEn11kE2ol9kv2Sl3tE/eqdR/CQdWWQ8iw6mqtphciYZLI2djtZf7BMitORcr0KJcA3xueuK7QhCKzUsHWnbzjFRIL40A9SpkdpeYAVH0IOMxkNJ2xl1PXirGUVyrfrEqT9IuXd082rJ859Pblkw5bDgmFtb8U7MPHVJLhF9vG1nw8IzUDn6mY+4qKhZr39HA0j3ziWJfw9UA+VO82Sa1FEqwfa6DZlpBJp1mbPt790QbKUDpvotUlJmRuwIMjugmqesIZqQx4eyThgg5TQKkm+06oOAbWhTmt45qiXvUZ7s0PwslmJd+vOEFzpYUIV+JINVv70xXe+ItNpQUTrpFnNBvjUGRpHZNqd7FE3k7KcACL+XVdPIgUZ9ebHoLWhGgqjAJWyZs1+lVR4CwIfLRC3ZsPOjnfymZQ7ARnQ100MQuG4LYrgoVi/JHHvlVM+QUBpD/S+6baQHPgz0/5ihs3mE3K0BTQ418/+Vhev1SBqKgqZHL8JG4otyFyPPWBY2hNkGvwqX8vMiynf2ubU5mMD12+8ieK9L9U4NTCTpjD0nBdDfu1GFuSrsJIvpG4d8cpgDff3QiLbZv5E/x68LeEIbeHzanx2ysI8v/K1GT+0errjVXbcnWz3bIbA8NZKo1TRKBbrebJDi7QNXAjekJ/yCoBOHUdBr6JzfcgaVzvyBgQQ04/EuKbh4v6ComOl9reFCS71koj37HK4y/gqR6KMhIMXTrac3of2IklmVtw/P6/Dna0dSj82gHnqb8EkhomxDLjeg4jO4ynGD5xaDXLTzY5KfUPp5KCi9+TnPCHepn+Jhg0zrwAiCF+pC7IPhw8WIJB4Cc6eK

In [105]:
int(time.time())

1662127068

In [42]:
columns = (
    'dt', 'symbol', 'open_price', 'high_price', 'low_price', 'close_price', 'total_volume'
)
print(columns)
print(values)

('dt', 'symbol', 'open_price', 'high_price', 'low_price', 'close_price', 'total_volume')
('2017-04-18', 'WVVIP', 'NaN', 'NaN', 'NaN', 5.5001, 140)


In [40]:
null_idxs = [i for i, value in enumerate(values) if value == 'NaN']
non_null_values = tuple([value for value in values if value != 'NaN'])
non_null_columns = '(' + ', '.join([column for i, column in enumerate(columns) if i not in null_idxs]) + ')'

In [41]:
print(non_null_values)
print(non_null_columns)

('2017-04-18', 'WVVIP', 5.5001, 140)
(dt, symbol, close_price, total_volume)


In [18]:
values[null_idxs]

TypeError: tuple indices must be integers or slices, not list

In [138]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)
session.row_factory = pandas_factory
session.default_fetch_size = None

result = session.execute(
    f''' 
    SELECT * FROM bakery.price_history_daily WHERE symbol = 'MSFT' ALLOW FILTERING;
    ''',
    timeout=None
)

df = result._current_rows

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   dt            0 non-null      object
 1   symbol        0 non-null      object
 2   close_price   0 non-null      object
 3   high_price    0 non-null      object
 4   low_price     0 non-null      object
 5   open_price    0 non-null      object
 6   total_volume  0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


In [136]:
df['dt'] = df['dt'].astype(str)

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854 entries, 0 to 1853
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   dt            1854 non-null   object 
 1   symbol        1854 non-null   object 
 2   close_price   1854 non-null   float64
 3   high_price    1854 non-null   float64
 4   low_price     1854 non-null   float64
 5   open_price    1854 non-null   float64
 6   total_volume  1854 non-null   int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 101.5+ KB


In [ ]:
df['dt'] = pd.to_datetime